In [12]:
import os
import numpy as np

import data_preprocessing
import neural_metrics

In [26]:
data_str = 'openimage'
binary_num_epochs = 4
binary_model_name = 'dinov2_vits14'

preprocessor = data_preprocessing.DataPreprocessor(data_str=data_str)

binary_results_path = 'binary_results'
binary_l_strs = {f
                 for f in os.listdir(binary_results_path)
                 if f.startswith(f'{data_str}_{binary_model_name}')}
len(binary_l_strs)

60

In [27]:
for name in binary_l_strs:
    curr_path = fr'{binary_results_path}/{name}'
    curr_data = np.load(curr_path)
    l_str = name.split(f'e{binary_num_epochs - 1}_')[-1].replace('.npy', '')
    is_fine = l_str in preprocessor.fine_grain_classes_str
    l_str_index = getattr(preprocessor, 
                          'fine_grain_classes_str' if is_fine else 'coarse_grain_classes_str').index(l_str) 
    test = len(curr_data) == len(preprocessor.test_true_coarse_data)
    true_values = (preprocessor.test_true_fine_data 
                   if is_fine else preprocessor.test_true_coarse_data) \
        if test else (preprocessor.train_true_fine_data 
                      if is_fine else preprocessor.train_true_coarse_data)
    binary_true = np.where(true_values == l_str_index, 1, 0)
    accuracy, f1, precision, recall = neural_metrics.get_individual_metrics(pred_data=curr_data,
                                                                            true_data=binary_true)
    print(f'class: {l_str}; '
          f'accuracy: {accuracy:.2f}, '
          f'f1: {f1:.2f}, '
          f'precision: {precision:.2f}, '
          f'recall: {recall:.2f}')
    
    assert f1 > 0.7

class: Clothing; accuracy: 0.95, f1: 0.94, precision: 0.94, recall: 0.93
class: Bicycle; accuracy: 0.98, f1: 0.88, precision: 0.87, recall: 0.90
class: Dessert; accuracy: 0.95, f1: 0.78, precision: 0.71, recall: 0.96
class: Monkey; accuracy: 1.00, f1: 0.98, precision: 0.98, recall: 0.99
class: Desk; accuracy: 0.96, f1: 0.78, precision: 0.72, recall: 0.88
class: Desk; accuracy: 0.97, f1: 0.85, precision: 0.78, recall: 0.96
class: Plant; accuracy: 0.95, f1: 0.87, precision: 0.86, recall: 0.88
class: Chair; accuracy: 0.97, f1: 0.82, precision: 0.76, recall: 0.95
class: Carnivore; accuracy: 0.98, f1: 0.88, precision: 0.86, recall: 0.90
class: Food; accuracy: 0.97, f1: 0.96, precision: 0.95, recall: 0.98
class: Vegetable; accuracy: 0.96, f1: 0.77, precision: 0.73, recall: 0.85
class: Suit; accuracy: 0.98, f1: 0.87, precision: 0.84, recall: 0.89
class: Swimwear; accuracy: 0.98, f1: 0.86, precision: 0.81, recall: 0.92
class: Flower; accuracy: 0.97, f1: 0.84, precision: 0.78, recall: 0.93
clas